# crawler_instagram

In [ ]:
pwd

### step 0. 준비

In [ ]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver import ActionChains as AC
from tqdm import tqdm
from tqdm import tqdm_notebook
import re
from time import sleep
import time

### step 1. 크롤링

In [ ]:
# 검색어
keyword = input("크롤링할 키워드를 입력하세요: ")
len_insta = ("크롤링할 글 수를 입력하세요: ")

In [ ]:
# 크롬창 띄우기
driver = webdriver.Chrome(r"G:\내 드라이브\exe\chromedriver.exe")
driver.get("https://www.instagram.com/")
time.sleep(2)

In [ ]:
dict = {}   # 전체 데이터를 담을 딕셔너리 생성

# 검색창에 커서 클릭
element = driver.find_element_by_css_selector(".pbgfb.Di7vw")
element.click()
time.sleep(1)

# 검색 창에 검색어 입력
element = driver.find_element_by_css_selector(".XTCLo.x3qfX")
element.send_keys(keyword)
time.sleep(3)

# 검색어 리스트 중 첫번째 검색어 클릭
query_list = driver.find_elements_by_css_selector(".-qQT3")
query_list[0].click()
time.sleep(7)

# 사진 클릭
CSS_tran="div.Nnq7C.weEfm"                                      # 사진 버튼 정의 
tran_button = driver.find_element_by_css_selector(CSS_tran)     # 사진 버튼 찾기
AC(driver).move_to_element(tran_button).click().perform()       # 사진 버튼 클릭
time.sleep(1)

# 크롤링 시작
for i in tqdm_notebook(range(0, len_insta)):

    target_info = {}                                            # 사진별 데이터를 담을 딕셔너리 생성

    try:
        # 사진(pic) 크롤링 시작
        overlays1 = "div._2dDPU.CkGkG .FFVAD"                   # 사진창 속 사진   
        img = driver.find_element_by_css_selector(overlays1)    # 사진 선택
        pic = img.get_attribute('src')                          # 사진 url 크롤링 완료
        target_info['picture'] = pic
        # print(target_info)

        # 날짜(date) 크롤링 시작
        overlays2 = "div._2dDPU.CkGkG .c-Yi7 > time"                # 날짜 지정
        datum2 = driver.find_element_by_css_selector(overlays2)     # 날짜 선택
        date = datum2.get_attribute('title')
        # get_attribute('title')                                    # 날짜 크롤링 완료
        target_info['date'] = date
        # print(target_info)
        # print(datalist)

        # 좋아요(like) 크롤링 시작
        overlays3 = ".Nm9Fw"                                        # 리뷰창 속 날짜
        datum3 = driver.find_element_by_css_selector(overlays3)     # 리뷰 선택
        like = datum3.text                                          # 좋아요 크롤링 완료
        target_info['like'] = like
        # print(target_info)

        # 해시태그(tag) 크롤링 시작
        overlays4 = ".C7I1f.X7jCj"                                  # 태그 지정
        datum3 = driver.find_element_by_css_selector(overlays4)     # 태그 선택
        tag_raw = datum3.text
        tags = re.findall('#[A-Za-z0-9가-힣]+', tag_raw)            # ""#OOO"만 뽑아오기(OOO: 한글,숫자,영어,_)
        tag = ''.join(tags).replace("#"," ")                        # "#" 제거
        target_info['tag'] = tag
        # print(target_info)

        dict[i] = target_info            # 토탈 딕셔너리로 만들기

        print("{0}".format(i), tag)

        # 다음장 클릭
        CSS_tran2="a._65Bje.coreSpriteRightPaginationArrow"             # 다음 버튼 정의
        tran_button2 = driver.find_element_by_css_selector(CSS_tran2)  # 다음 버튼 find
        AC(driver).move_to_element(tran_button2).click().perform()     # 다음 버튼 클릭
        time.sleep(2)

    except:  # 에러가 나면
        # 다음장 클릭
        CSS_tran2="a._65Bje.coreSpriteRightPaginationArrow"             # 다음 버튼 정의
        tran_button2 = driver.find_element_by_css_selector(CSS_tran2)  # 다음 버튼 find
        AC(driver).move_to_element(tran_button2).click().perform()     # 다음 버튼 클릭
        time.sleep(2)

driver.close()

print(dict)

# 판다스화
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')

n = result_df['picture'].count()

# 엑셀 저장
result_df.to_excel("insta_({}).xlsx".format(keyword), encoding='euc-kr')

In [ ]:
result_df

In [ ]:
num_pic = len(result_df['picture'])
num_pic

### step 2. 이미지 저장

In [ ]:
# 이미지들 image_insta 폴더에 다운받기
import os
import urllib.request

# 만약 폴더가 없으면 생성
if not os.path.exists("image_insta_"+keyword):  #if not () 없으면
    os.makedirs("image_insta_"+keyword)         # 생성

# 이미지 저장
for i in range(0, 50):
    try:
        index = result_df['picture'][i]
        date = result_df['date'][i]
        urllib.request.urlretrieve(index, "image_insta_{0}/{1}_{2}.jpg".format(keyword, i, date))        
    except:
        pass